# D3 v4 with Jupyter

This notebook shows the process of displaying a d3 v4 graph widget in a Jupyter notebook using data that lives inside a Pandas DataFrame. I orginally followed the process detailed [here](https://github.com/stitchfix/d3-jupyter-tutorial) by stitchfix. Its very well done but unfortunately it doesn't seem to work work for d3 v4. Calling d3 v3 using `HTML('<script src="lib/d3/d3.min.js"></script>')` is fine, but v4 is not loading with require.js.

I wanted to get this working with d3 v4 and also using the `Javascript` display function rather than pushing in a script tag through the `HTML` display function. This it not the most exciting example ever, but it gets the process of moving data in a Pandas DataFrame into the DOM and accessible by D3. You can then edit your data queries and modify the output that D3 is displaying in one environment.


In [1]:
from IPython.display import Javascript,HTML
import json
import seaborn as sns
import pandas

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# This will use the Iris dataset that seems to accompany most R examples. Its part of seaborn. Handy!
iris = sns.load_dataset('iris')

In [3]:
iris.sample(10)

,sepal_length,sepal_width,petal_length,petal_width,species
108,6.7,2.5,5.8,1.8,virginica
59,5.2,2.7,3.9,1.4,versicolor
43,5.0,3.5,1.6,0.6,setosa
125,7.2,3.2,6.0,1.8,virginica
132,6.4,2.8,5.6,2.2,virginica
79,5.7,2.6,3.5,1.0,versicolor
37,4.9,3.6,1.4,0.1,setosa
67,5.8,2.7,4.1,1.0,versicolor
60,5.0,2.0,3.5,1.0,versicolor
143,6.8,3.2,5.9,2.3,virginica


In [4]:
# There are only 3 species types in the data, so colors of the bars are assigned with `d3.schemeCategory10`
iris['species'].drop_duplicates().tolist()

['setosa', 'versicolor', 'virginica']

Editing javascript a Python notebook is not great. You don't get syntax highlighting, code completion or other handy stuff that a good editor will do. I use Sublime Text and had a file that I edited and re-loaded using the code below.

In [5]:
#Javascript(filename='some_js_file.js')

## The Actual Javascript
This was trickier than it should be as the functions and variables aren't accessible to other `Javascript` instances unless you use the `window.` prefix. This is one function `udpater` that does all the work. Since the `svg` variable also not accessiable outside this function its necessary to check if it existing on each iteration and create it on the first one. 

The d3 stuff was quite straight forware. This uses the new d3 v4 enter-update-exit pattern described [here](https://bl.ocks.org/mbostock/3808218). I also grabbed some code from [here](https://bl.ocks.org/d3noob/bdf28027e0ce70bd132edc64f1dd7ea4). 

In [6]:
Javascript("""
require.config({
    paths: {
        d3: "https://d3js.org/d3.v4.min"
    }
});

require(["d3"], function(d3) {

    window.updater = function(data) {
        d3.selectAll("p").html("Changed <b>content</b>")

        var svg_margin  = { top: 20, right: 20, bottom: 20, left: 40 };
        var svg_width   = 960 - svg_margin.left - svg_margin.right;
        var svg_height  = 500 - svg_margin.top - svg_margin.bottom;

        var y = d3.scaleLinear()
            .domain([0, d3.max(data, function(d) { return d.petal_length; })])
            .range([svg_height, 0]);

        var x = d3.scaleBand()
            .domain(d3.range(data.length))
            .range([0, svg_width])
            .padding(0.1);

        var species_list = d3.map(data, function (d) { return d.species;}).keys();

        if (d3.select("#svg_container").select("svg").empty()) {

            window.svg = d3.select("#svg_container").append("svg")
                .attr("width", svg_width + svg_margin.left + svg_margin.right)
                .attr("height", svg_height + svg_margin.top + svg_margin.bottom)
                .append("g")
                .attr("transform",
                    "translate(" + svg_margin.left + "," + svg_margin.top + ")");

            svg.append("g")
                .attr("transform", "translate(0," + svg_height + ")")
                .attr("class", "x axis")
                .call(d3.axisBottom(x));

            // add the y Axis
            svg.append("g")
                .attr("class", "y axis")
                .call(d3.axisLeft(y));
        } else {
            svg.selectAll("g.y.axis")
                .call(d3.axisLeft(y));

            svg.selectAll("g.x.axis")
                .call(d3.axisBottom(x));
        }

        // DATA JOIN
        // Join new data with old elements, if any.

        var bars = svg.selectAll(".bar")
            .data(data);

        // UPDATE
        // Update old elements as needed.

        bars
            .attr("style",function(d) { return "fill:" + d3.schemeCategory10[species_list.indexOf(d.species)];})
            .attr("x", function(d, i) { return x(i); })
            .attr("width", x.bandwidth())
            .transition()
            .duration(100)
            .attr("y", function(d) { return y(d.petal_length); })
            .attr("height", function(d) { return svg_height - y(d.petal_length); });

        // ENTER + UPDATE
        // After merging the entered elements with the update selection,
        // apply operations to both.

        bars.enter().append("rect")
            .attr("class", "bar")
            .attr("style",function(d) { return "fill:" + d3.schemeCategory10[species_list.indexOf(d.species)];})
            .attr("x", function(d, i) { return x(i); })
            .attr("width", x.bandwidth())
            .attr("y", function(d) { return y(d.petal_length); })
            .attr("height", function(d) { return svg_height - y(d.petal_length); })
            .merge(bars);

        // EXIT
        // Remove old elements as needed.

        bars.exit().remove();

    };
});
""")

<IPython.core.display.Javascript object>

## The HTML
This needs some HTML scafholding for d3 to select. You can add more complex styling, HTML inputs etc here and also access it via a file. 

In [7]:
HTML("""
<style>

.axis {
  font: 12px sans-serif;
}
</style>

<body>
<span> Cluster data </span>
<p class="main-paragraph">1. Line </p>
<p>2. line</p>
</body>

<div id='svg_container' class=""></div>
""")


## Binding the Data
Re-run the code below to update the graph. You can change the sample size to add or remove bars from the graph.

In [8]:
# This selects a random sample of 10 rows from the Iris DataFrom. Its push to the DOM in d3 specific format by 
# converting the data frame to a dictionary, oriented by `records`. At this point is just a long, correctly formatted
# string being run inside the browser that creates an oject that d3 can work with. 

record_set = json.dumps(iris.sample(10).to_dict(orient='records'))
#[print(item) for item in record_set.split('},')]

Javascript("""
require(["d3"], function(d3) {{
    updater({})
}});
""".format(record_set))

<IPython.core.display.Javascript object>

## Registering the d3 module to use on the console
Occasionally you are going to need to debug things on the console, and if your run the code snippet below, it adds d3 as a global function.

In [9]:
Javascript("""
require(["d3"], function(d3) {
  window.d3 = d3;
});
""")

<IPython.core.display.Javascript object>